<a href="https://colab.research.google.com/github/alibelhrak/python_chatbot_scratch/blob/main/RNN_Healthcare_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout   , Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
import nltk
from nltk.stem import WordNetLemmatizer


In [ ]:
import json

    with open('/content/intents.json') as f:
        intents_data = json.load(f)

    with open('/content/intents_data.json', 'w') as f:
        json.dump(intents_data, f) # intents_data is correctly used here


IndentationError: unexpected indent (<ipython-input-17-1aee4cc18673>, line 3)

In [ ]:
intents_data_json

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'isn't', 'patterns': ['this isn't work..."
4,"{'tag': 'hours', 'patterns': ['What hours are ..."
...,...
715,"{'tag': 'anger disorder', 'patterns': ['i cons..."
716,"{'tag': 'anxiety anger', 'patterns': ['i const..."
717,"{'tag': 'abruptly', 'patterns': ['hello i'm 20..."
718,"{'tag': 'anger outbursts', 'patterns': ['hello..."


In [ ]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
words=[]
classes=[]
documents=[]
ignore_symbols = ['.','?' , ';' , ':' , '!' , '%']
from nltk.tokenize import word_tokenize

for intent in intents_data_json["intents"]:
    for pattern in intent["patterns"]:
        words_list = word_tokenize(pattern)
        words.append(words_list)
        documents.append((words_list, intent["tag"]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

print("Documents:", documents)

Documents: [(['Hello'], 'greeting'), (['Hi'], 'greeting'), (['Hey'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['Good', 'morning'], 'greeting'), (['Bye'], 'goodbye'), (['Goodbye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Take', 'care'], 'goodbye'), (['Thank', 'you'], 'thanks'), (['Thanks'], 'thanks'), (['Appreciate', 'it'], 'thanks'), (['Can', 'you', 'help', 'me', '?'], 'help'), (['I', 'need', 'assistance'], 'help'), (['Help', 'me'], 'help'), (['What', "'s", 'the', 'weather', 'like', '?'], 'weather'), (['Is', 'it', 'raining', 'today', '?'], 'weather'), (['Tell', 'me', 'the', 'forecast'], 'weather'), (['Tell', 'me', 'a', 'joke'], 'joke'), (['Make', 'me', 'laugh'], 'joke'), (['Do', 'you', 'know', 'any', 'jokes', '?'], 'joke')]


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
words = [lemmatizer.lemmatize(word) for sublist in words for word in sublist if word not in ignore_symbols]
words = list(sorted(set(words)))
classes = list(sorted(set(classes)))


In [ ]:
import pickle

with open('words.pkl' , 'wb') as f:
  pickle.dump(words , f)

with open('classes.pkl' , 'wb') as f:
  pickle.dump(classes , f)

In [ ]:
import random
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag=[]
  word_patterns = document[0]
  word_patterns =[lemmatizer.lemmatize(word) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  outputrow = list(output_empty)
  outputrow[classes.index(document[1])] = 1
  training.append([bag  , outputrow])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[: , 0])
train_y = list(training[: , 1])

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(train_x[0]),)))
model.add(Dropout(0.25))
model.add(Dense(64 , activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(len(train_y[0]) , activation = 'softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd ,  metrics=['accuracy'] , loss='categorical_crossentropy' )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(np.array(train_x) , np.array(train_y) , epochs=200 , batch_size =5 , verbose=0)

In [ ]:
import random
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Assuming lemmatizer, words, classes, and model are already defined
lemmatizer = WordNetLemmatizer()
ignore_symbols = ['?', '!', '.', ',', "'s", "'m"]

def clean_up(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_symbols]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    predicted_word = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.1  # Adjusted threshold

    results = [[i, r] for i, r in enumerate(predicted_word) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"DEBUG: Results after threshold: {results}")  # Debugging print

    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})

    print(f"DEBUG: Return list: {return_list}")  # Debugging print

    return return_list

def get_responses(intent_list, intent_json):
    tag = intent_list[0]['intent']
    list_of_intents = intent_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Assuming intents_data is already loaded as a dictionary
while True:
    message = input("Vous: ")

    if message.lower() in ['quit', 'exit', 'goodbye']:
        print("Chatbot: Thanks for this conversation! Goodbye 😊")
        break

    ints = predict_class(message)

    if not ints:
        print("Chatbot: Sorry, this message contains things beyond my knowledge!")
    else:
        res = get_responses(ints, intents_data)
        print("Chatbot:", res)